In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/googleapis/langchain-google-cloud-sql-pg-python/tree/main/docs/example_app.ipynb)

---
# **Introduction**

In this codelab you will learn how to create an interactive generative AI application with Retrieval Augmented Generation using Cloud SQL Postgres and langchain. We will be creating an application grounded in a Netflix Movie dataset.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic python knowledge

## What you'll learn

* How to deploy a Cloud SQL Postgres instance
* How to use Cloud SQL Postgres as a vector store
* How to use Cloud SQL Postgres as a document loader
* How to use Cloud SQL Postgres for chat history storage

## What you'll need
* A Google Cloud Account and Google Cloud Project
* A web browser such as [Chrome](https://www.google.com/chrome/)

# **Setup and Requirements**

In the following instructions you will learn to:
1. Install required dependencies for our application
2. Set up authentication for our project
3. Set up a Cloud SQL Postgres Instance
4. Import the data used by our application

## Install dependencies
Installing the dependencies needed to run this demo app.

In [ ]:
# TODO: Add all dependencies here
# Update this to install from Pypi
!pip install git+https://github.com/googleapis/langchain-google-cloud-sql-pg-python.git
# !pip install langchain_google_cloud_sql_pg

!pip install langchain_core, langchain

## Authenticate to Google Cloud within Colab
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

In [8]:
from google.colab import auth

auth.authenticate_user()

## Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook so that you can leverage Google Cloud from within Colab.

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "hailongli-playground2"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

## Configure Your Google Cloud Project
Configure the following in your Google Cloud Project.

1. IAM principal (user, service account, etc.) with the
[Cloud SQL Client][client-role] role.

> The user logged into this notebook will be used as the IAM principal and will be granted the Cloud SQL Client role.

[client-role]: https://cloud.google.com/sql/docs/mysql/roles-and-permissions

In [ ]:
# grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

2. Enable the [Cloud SQL Admin API][admin-api] within your project.

[admin-api]: https://console.cloud.google.com/apis/api/sqladmin.googleapis.com

In [ ]:
# enable GCP services
!gcloud services enable sqladmin.googleapis.com
!gcloud services enable aiplatform.googleapis.com

## Setting up Cloud SQL
A **Postgres** Cloud SQL instance is required for the following stages of this notebook.

### Create a Postgres Instance
Running the below cell will verify the existence of the Cloud SQL instance and or create a new instance and database if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [ ]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1" #@param {type:"string"}
instance_name = "codelab-pg-langchain-2" #@param {type:"string"}
database_name = "langchain-test" #@param {type:"string"}
password = input("Please provide a password to be used for 'postgres' database user: ")

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide the name of your database_name"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_14 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql.iam_authentication=On
  !gcloud sql databases create {database_name} --instance={instance_name}


## Import data to your database

> Indented block



Now that you have your database, you will need to import data! We will be using a Netflix Dataset which can be accessed [here](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here are some example data from the csv.

| show_id | type    | title                          | director                    | cast                                                      | country                                            | date_added        | release_year | rating | duration | listed_in                                   | description                                                                                     |
|---------|---------|--------------------------------|-----------------------------|-----------------------------------------------------------|----------------------------------------------------|-------------------|--------------|--------|----------|---------------------------------------------|-------------------------------------------------------------------------------------------------|
| s6       | TV Show | Midnight Mass                  | Mike Flanagan               | Kate Siegel, Zach Gilford, Hamish Linklater, H...         | None                                               | September 24, 2021 | 2021         | TV-MA  | 1 Season | TV Dramas, TV Horror, TV Mysteries         | The arrival of a charismatic young priest brin...                                                |
| s7       | Movie   | My Little Pony: A New Generation | Robert Cullen, José Luis Ucha | Vanessa Hudgens, Kimiko Glenn, James Marsden, ...         | None                                               | September 24, 2021 | 2021         | PG     | 91 min   | Children & Family Movies                   | Equestria's divided. But a bright-eyed hero be...                                                |
| s8       | Movie   | Sankofa                        | Haile Gerima                | Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...         | United States, Ghana, Burkina Faso, United Kin... | September 24, 2021 | 1993         | TV-MA  | 125 min  | Dramas, Independent Movies, International Movies | On a photo shoot in Ghana, an American model s...                                                |
| s9       | TV Show | The Great British Baking Show  | Andy Devonshire             | Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...         | United Kingdom                                     | September 24, 2021 | 2021         | TV-14  | 9 Seasons| British TV Shows, Reality TV                | A talented batch of amateur bakers face off in...                                                |
| s10       | Movie   | The Starling                   | Theodore Melfi              | Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...         | United States                                      | September 24, 2021 | 2021         | PG-13  | 104 min  | Comedies, Dramas                            | A woman adjusting to life after a loss contend...                                                |


You won't need to directly load the csv data into your database. Instead we prepared a table, "netflix_titles", in the format of a `.sql` file for Postgres. You can easily import the table into your database with one `gcloud` command.

In [ ]:
# Adding the IAM User to the database for the import operation
!gcloud sql users create {current_user[0]} --instance={instance_name} --type=CLOUD_IAM_USER

In [ ]:
# Import the netflix titles table using gcloud command
import_command_output = !gcloud sql import sql {instance_name} gs://cloud-samples-data/langchain/postgres/netflix_titles.sql --database={database_name} --quiet

if "Imported data" in str(import_command_output):
    print(import_command_output)
elif "already exists" in str(import_command_output):
    print("Did not import because the table already existed.")
else:
    raise Exception(f"The import seems failed:\n{import_command_output}")

# **Use case 1: Cloud SQL Postgres as a document loader**

Now that you have data in your database, you are ready to use Cloud SQL Postgres as a document loader. This means we will pull data from the database and load it into memory as documents. This will let us feed documents into the vector store.

First lets install the core langchain library so that we can import the Document class to use when preparing documents to feed into our vector store.

In [ ]:
!pip install langchain_core

Next let's connect to our CloudSQL PostgreSQL instance using the PostgreSQLEngine class.

In [14]:
from langchain_google_cloud_sql_pg import PostgreSQLLoader, PostgreSQLEngine, Column

pg_engine = PostgreSQLEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user="postgres",
    password=password,
)

Once we initialize a PostgreSQLEngine object, we can pass it into the PostgreSQLLoader to connect to a specific database. As you can see we also pass in a query, table_name and a list of columns. The query tells the loader what query to use to pull data. The "content_columns" argument refers to the columns that will be used as "content" in the document object we will later construct. The rest of the columns in that table will become the "metadata" associated with the documents.

In [15]:
table_name = "netflix_titles"
content_columns = ["title", "director", "cast", "description"]
loader = PostgreSQLLoader(
    engine=pg_engine,
    query=f"SELECT * FROM {table_name};",
    content_columns=content_columns,
)

Next let's define a function "collect_async_items" to asynchronously pull documents from our database.

In [16]:
async def collect_async_items(docs_generator):
    """Collects items from an async generator."""
    docs = []
    async for doc in docs_generator:
        docs.append(doc)
    return docs

Then let's run the function to pull our documents from out database using our document loader. You can see the first 5 documents from the database here. Nice, you just used CloudSQL Postgres as a document loader!

In [17]:
documents = await collect_async_items(loader.alazy_load())
print(f"Loaded {len(documents)} from the database. 5 Examples:")
for doc in documents[:5]:
    print(doc)

Loaded 8802 from the database. 5 Examples:
page_content='Midnight Mass Mike Flanagan Kate Siegel, Zach Gilford, Hamish Linklater, Henry Thomas, Kristin Lehman, Samantha Sloyan, Igby Rigney, Rahul Kohli, Annarah Cymone, Annabeth Gish, Alex Essoe, Rahul Abburi, Matt Biedel, Michael Trucco, Crystal Balint, Louis Oliver The arrival of a charismatic young priest brings glorious miracles, ominous mysteries and renewed religious fervor to a dying town desperate to believe.' metadata={'show_id': 's6', 'type': 'TV Show', 'country': None, 'date_added': 'September 24, 2021', 'release_year': 2021, 'rating': 'TV-MA', 'duration': '1 Season', 'listed_in': 'TV Dramas, TV Horror, TV Mysteries'}
page_content="My Little Pony: A New Generation Robert Cullen, José Luis Ucha Vanessa Hudgens, Kimiko Glenn, James Marsden, Sofia Carson, Liza Koshy, Ken Jeong, Elizabeth Perkins, Jane Krakowski, Michael McKean, Phil LaMarr Equestria's divided. But a bright-eyed hero believes Earth Ponies, Pegasi and Unicorns sho

# **Use case 2: Cloud SQL Postgres as Vector Store**

Now, let's learn how to put all of the documents we just loaded into a vector store so that we can use vector search to answer our user's questions!

### Create Your Vector Store table

Based on the documents that we loaded before, we want to create a table with a vector column as our vector store. We will start it by intializing a vector table by calling the `init_vectorstore_table` function from our `engine`. As you can see we list all of the columns for our metadata. We also specify a vector size, 768, that corresponds with the length of the vectors computed by the model our embeddings service uses, Vertex AI's textembedding-gecko.


In [18]:
from langchain_google_cloud_sql_pg import PostgreSQLEngine, Column

sample_vector_table_name = "movie_vector_table_samples"

pg_engine = PostgreSQLEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user="postgres",
    password=password,
)

await pg_engine.init_vectorstore_table(
    sample_vector_table_name,
    vector_size=768,
    metadata_columns=[
        Column("show_id", "VARCHAR", nullable=True),
        Column("type", "VARCHAR", nullable=True),
        Column("country", "VARCHAR", nullable=True),
        Column("date_added", "VARCHAR", nullable=True),
        Column("release_year", "INTEGER", nullable=True),
        Column("rating", "VARCHAR", nullable=True),
        Column("duration", "VARCHAR", nullable=True),
        Column("listed_in", "VARCHAR", nullable=True),
    ],
    store_metadata=True,
    overwrite_existing=True,
)

Before we continue, we will need to install the Vertex AI langchain library as we will are using Vertex AI as our embeddings service.

In [ ]:
!pip install langchain_google_vertexai

### Try load the documents to the vector table

Now we will create a vector_store object backed by our vector table in the Cloud SQL database. Let's load the data from the documents to the vector table. Note that for each row, the embedding service will be called to compute the embeddings to store in the vector table. Pricing details can be found [here](https://cloud.google.com/vertex-ai/pricing).

In [19]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresVectorStore, PostgreSQLEngine

# Initialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=project_id
)

pg_engine = PostgreSQLEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user="postgres",
    password=password,
)

vector_store = PostgresVectorStore.create_sync(
    engine=pg_engine,
    embedding_service=embeddings_service,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

Now let's try to put the documents data into the vector table. Here is a code example to load the first 5 documents in the list.

In [ ]:
import uuid

docs_to_load = documents[:5]

# ! Uncomment the following line to load all 8,800+ documents to the database vector table with calling the embedding service.
# docs_to_load = documents

ids = [str(uuid.uuid4()) for i in range(len(docs_to_load))]
vector_store.add_documents(docs_to_load, ids)

### Import the rest of your data into your vector table

You don't have to call the embedding service 8,800 times to load all the documents for the demo. Instead, we have prepared a table with the all 8,800+ rows with pre-computed embeddings in a `.sql` file. Again, let's import to our DB using `gcloud` command.

It will restore the `.sql` file to a table with vectors called `movie_vector_table`.

In [ ]:
# Import the netflix titles with vector table using gcloud command
import_command_output = !gcloud sql import sql {instance_name} gs://cloud-samples-data/langchain/postgres/netflix_titles_vector_table.sql --database={database_name} --quiet

if "Imported data" in str(import_command_output):
    print(import_command_output)
elif "already exists" in str(import_command_output):
    print("Did not import because the table already existed.")
else:
    raise Exception(f"The import seems failed:\n{import_command_output}")

# **Use case 3: Cloud SQL Postgres as Chat Memory**

Next we will add chat history (called “memory” in the context of LangChain) to our application so the LLM can retain context and information across multiple interactions, leading to more coherent and sophisticated conversations or text generation. We can use Cloud SQL Postgres as “memory” storage in our application so that the LLM can use context from prior conversations to better answer the user’s prompts. First let's initialize Cloud SQL Postgres as memory storage.

In [22]:
from langchain_google_cloud_sql_pg import PostgreSQLChatMessageHistory, PostgreSQLEngine

pg_engine = PostgreSQLEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database="langchain-test",
    user="postgres",
    password=password,
)

message_table_name = "message_store"

pg_engine.run_as_sync(pg_engine.init_chat_history_table(table_name=message_table_name))

chat_history = PostgreSQLChatMessageHistory(
    pg_engine,
    session_id="my-test-session",
    table_name=message_table_name,
)

Here is an example of how you would add a user message and how you would add an ai message.

In [23]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

[HumanMessage(content='What movie was Brad Pitt in?'),
 AIMessage(content='Brad Pitt was in Inglourious Basterds, By the Sea, Killing Them Softly and Babel according to the given data.'),
 HumanMessage(content='How about Jonny Depp?'),
 AIMessage(content="Jonny Depp was in Charlie and the Chocolate Factory, The Rum Diary, The Imaginarium of Doctor Parnassus, and What's Eating Gilbert Grape according to the given data."),
 HumanMessage(content='Are there movies about animals?'),
 AIMessage(content='Yes, Rango features animals.'),
 HumanMessage(content='Hi!'),
 AIMessage(content="Hello there. I'm a model and am happy to help!")]

# **Conversational RAG Chain backed by Cloud SQL Postgres**

So far we've tested with using Cloud SQL Postgres as document loader, Vector Stoe and Chat Memory. Now let's use it in the `ConversationalRetrievalChain`.

We will build a chat bot that can answer movie related questions based on the vector search results.

First let's initialize all of our PostgresSQLEngine object to use as a connection in our vector store and chat_history.

In [ ]:
!pip install langchain

In [26]:
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_google_cloud_sql_pg import (
    PostgreSQLEngine,
    PostgresVectorStore,
    PostgreSQLChatMessageHistory,
)

# Intialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=project_id
)

# Intialize the engine
pg_engine = PostgreSQLEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database="langchain-test",
    user="postgres",
    password=password,
)

# Intialize the Vector Store
vector_table_name = "movie_vector_table"
vector_store = PostgresVectorStore.create_sync(
    engine=pg_engine,
    embedding_service=embeddings_service,
    table_name=vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

# Intialize the PostgreSQLChatMessageHistory
chat_history = PostgreSQLChatMessageHistory(
    pg_engine,
    session_id="my-test-session",
    table_name="message_store",
)

Let's create a prompt for the LLM. Here we can add instructions specific to our application, such as "Don't make things up".

In [27]:
# Prepare some prompt templates for the ConversationalRetrievalChain
prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

Now let's use our vector store as a retreiver. Retreiver's in Langchain allow us to literally "retrieve" documents.

In [28]:
# Intialize retriever, llm and memory for the chain
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

Now let's intialize our LLM, in this case we are using Vertex AI's "gemini-pro".

In [29]:
llm = VertexAI(model_name="gemini-pro", project=project_id)

We clear our chat history, so that our application starts without any prior context to other conversations we have had with the application.

In [30]:
chat_history.clear()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

Now let's create a conversational retrieval chain. This will allow the LLM to use chat history in it's responses, meaning we can ask it follow up questions to our questions instead of having to start from scratch after each inquiry.

In [32]:
# create the ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# ask some questions
q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Jonny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

# browser the chat history
chat_history.messages

Question: What movie was Brad Pitt in?
Answer: Inglourious Basterds, By the Sea, Killing Them Softly, Babel

Question: How about Jonny Depp?
Answer: Charlie and the Chocolate Factory, The Rum Diary, The Imaginarium of Doctor Parnassus, What's Eating Gilbert Grape (Vector search result)

Question: Are there movies about animals?
Answer: Yes, there are movies about animals. For example, "Animals on the Loose: A You vs. Wild Movie" is an interactive special where you and Bear Grylls must pursue escaped wild animals and secure their protective habitat. (Vector search result)



[HumanMessage(content='What movie was Brad Pitt in?'),
 AIMessage(content='Inglourious Basterds, By the Sea, Killing Them Softly, Babel'),
 HumanMessage(content='How about Jonny Depp?'),
 AIMessage(content="Charlie and the Chocolate Factory, The Rum Diary, The Imaginarium of Doctor Parnassus, What's Eating Gilbert Grape (Vector search result)"),
 HumanMessage(content='Are there movies about animals?'),
 AIMessage(content='Yes, there are movies about animals. For example, "Animals on the Loose: A You vs. Wild Movie" is an interactive special where you and Bear Grylls must pursue escaped wild animals and secure their protective habitat.'),
 HumanMessage(content='What movie was Brad Pitt in?'),
 AIMessage(content='Inglourious Basterds, By the Sea, Killing Them Softly, Babel'),
 HumanMessage(content='How about Jonny Depp?'),
 AIMessage(content="Charlie and the Chocolate Factory, The Rum Diary, The Imaginarium of Doctor Parnassus, What's Eating Gilbert Grape (Vector search result)"),
 Human